In [1]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset

dataset = SingleInputDataset.from_csv("../../final_data/test_300.csv", representation_field="sequence", instances_ids_field="accession", labels_field=slice(8, 2779))

/home/jcapela/PlantsSM/src/plants_sm/io/csv.py:56: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self.path, **self.kwargs)


In [2]:
from plants_sm.data_standardization.truncation import Truncator
dataset = Truncator(max_length=800).fit_transform(dataset)

Truncator: 100%|██████████| 300/300 [00:00<00:00, 42826.70it/s]


In [1]:
from plants_sm.models.ec_number_prediction.esm import EC_ESM_Lightning

model = EC_ESM_Lightning("esm2_t12_35M_UR50D", [2560, 5120], 2771)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jcapela/miniforge3/envs/plants_sm/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
from plants_sm.featurization.proteins.bio_embeddings.constants import ESM_LAYERS


layers_1 = ESM_LAYERS["esm2_t12_35M_UR50D"] - 1
layer_2 = ESM_LAYERS["esm2_t12_35M_UR50D"] - 2
no_grad = set()
for parameter in model.named_parameters():
    if "layers" in parameter[0] and str(layers_1) not in parameter[0] and str(layer_2) not in parameter[0]:
        no_grad.add(parameter[0])
no_grad

{'esm_model.layers.0.fc1.bias',
 'esm_model.layers.0.fc1.weight',
 'esm_model.layers.0.fc2.bias',
 'esm_model.layers.0.fc2.weight',
 'esm_model.layers.0.final_layer_norm.bias',
 'esm_model.layers.0.final_layer_norm.weight',
 'esm_model.layers.0.self_attn.k_proj.bias',
 'esm_model.layers.0.self_attn.k_proj.weight',
 'esm_model.layers.0.self_attn.out_proj.bias',
 'esm_model.layers.0.self_attn.out_proj.weight',
 'esm_model.layers.0.self_attn.q_proj.bias',
 'esm_model.layers.0.self_attn.q_proj.weight',
 'esm_model.layers.0.self_attn.v_proj.bias',
 'esm_model.layers.0.self_attn.v_proj.weight',
 'esm_model.layers.0.self_attn_layer_norm.bias',
 'esm_model.layers.0.self_attn_layer_norm.weight',
 'esm_model.layers.1.fc1.bias',
 'esm_model.layers.1.fc1.weight',
 'esm_model.layers.1.fc2.bias',
 'esm_model.layers.1.fc2.weight',
 'esm_model.layers.1.final_layer_norm.bias',
 'esm_model.layers.1.final_layer_norm.weight',
 'esm_model.layers.1.self_attn.k_proj.bias',
 'esm_model.layers.1.self_attn.k_pr

In [12]:
for parameter in model.named_parameters():
    print(parameter[0])

esm_model.embed_tokens.weight
esm_model.layers.0.self_attn.k_proj.weight
esm_model.layers.0.self_attn.k_proj.bias
esm_model.layers.0.self_attn.v_proj.weight
esm_model.layers.0.self_attn.v_proj.bias
esm_model.layers.0.self_attn.q_proj.weight
esm_model.layers.0.self_attn.q_proj.bias
esm_model.layers.0.self_attn.out_proj.weight
esm_model.layers.0.self_attn.out_proj.bias
esm_model.layers.0.self_attn_layer_norm.weight
esm_model.layers.0.self_attn_layer_norm.bias
esm_model.layers.0.fc1.weight
esm_model.layers.0.fc1.bias
esm_model.layers.0.fc2.weight
esm_model.layers.0.fc2.bias
esm_model.layers.0.final_layer_norm.weight
esm_model.layers.0.final_layer_norm.bias
esm_model.layers.1.self_attn.k_proj.weight
esm_model.layers.1.self_attn.k_proj.bias
esm_model.layers.1.self_attn.v_proj.weight
esm_model.layers.1.self_attn.v_proj.bias
esm_model.layers.1.self_attn.q_proj.weight
esm_model.layers.1.self_attn.q_proj.bias
esm_model.layers.1.self_attn.out_proj.weight
esm_model.layers.1.self_attn.out_proj.bia

In [4]:
from plants_sm.models.language_pytorch_models import EC_ESM1bLightningModel

model_l = EC_ESM1bLightningModel(model, max_epochs=2, devices=[1, 2], batch_size=2, strategy="ddp_notebook",
                               accelerator="gpu")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
tokens = model_l.preprocess(dataset)

In [6]:
tokens

In [5]:
model_l.fit(dataset)

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/jcapela/miniforge3/envs/plants_sm/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/jcapela/PlantsSM/src/plants_sm/parallelisation/torch_spawner.py", line 127, in distributed_worker
    results = main_func(**kwargs)
  File "/home/jcapela/miniforge3/envs/plants_sm/lib/python3.9/site-packages/lightning/pytorch/trainer/trainer.py", line 544, in fit
    call._call_and_handle_interrupt(
  File "/home/jcapela/miniforge3/envs/plants_sm/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py", line 43, in _call_and_handle_interrupt
    return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer, **kwargs)
  File "/home/jcapela/miniforge3/envs/plants_sm/lib/python3.9/site-packages/lightning/pytorch/strategies/launchers/multiprocessing.py", line 107, in launch
    _check_bad_cuda_fork()
  File